In [1]:
# When using Colab, make sure you run this instruction beforehand
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 5.7 MB/s eta 0:00:00


1) **Strength of the ILP formulation with cut-set inequalities.**

Let $G = (V,A)$ be a complete directed graph, with a cost $c_{ij} \in \mathbb{R}$ for each arc $(i, j) \in A$.
Consider the following Integer Linear Programming formulation (Dantzig, Fulkerson, Johnson,
1959) for the ATSP:



\begin{array}{lll}
  (DFJ) &  \min & \sum\limits_{(i,j) \in A} c_{ij}x_{ij} \qquad & (1) & \\
  & \textrm{s.t.} &  & \\
  & \sum\limits_{j\in V: j \neq i } x_{ij} = 1 &  i \in V & (2) &\\
  & \sum\limits_{i\in V: j \neq i } x_{ij} = 1 &  j \in V & (3) &\\
  & \sum\limits_{(i,j) \in \delta^+(S) } x_{ij} \geq 1 &  S \subset V, |S|>1 & (4) &\\
  & x_{ij} \in \{0,1\} &  (i,j) \in A &  (5)&\\
\end{array}

where the constraints (4) are the $\textit{cut-set inequalities}$, and its linear relaxation:


\begin{array}{lll}
  (DFJ^0) &  \min & \sum\limits_{(i,j) \in A} c_{ij}x_{ij} \qquad & (6) & \\
  & \textrm{s.t.} &  & \\
  & \sum\limits_{j\in V: j \neq i } x_{ij} = 1 &  i \in V & (7) &\\
  & \sum\limits_{i\in V: j \neq i } x_{ij} = 1 &  j \in V & (8) &\\
  & \sum\limits_{(i,j) \in \delta^+(S) } x_{ij} \geq 1 &  S \subset V, |S|>1 & (9) &\\
  & x_{ij} \geq 0 &  (i,j) \in A & (10) &\\
\end{array}

a) Write in Python the original formulation $(DFJ)$ and its linear relaxation $(DFJ^0)$.

In [2]:
import mip
import string
import ast
import pandas as pd # to handle the data of the problem
from itertools import chain, combinations # for the computation of the powerset
from mip import BINARY
import numpy as np

In [3]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

In [35]:
# SET DEFINITION

# Set of nodes
g = pd.read_csv('V.csv',sep=';')
V = list(g['index'])

# Cardinality of set V
n = len(V)


# PARAMETER DEFINITION

# Cost of each edge
temp_c = pd.read_csv('c.csv',sep=';')
temp_c['index'] = temp_c['index'].apply(ast.literal_eval)
temp_c.set_index('index', inplace = True)
c = temp_c.to_dict()['c']

# Auxiliary set A

A = list(c.keys())

S = list(powerset(V))

In [36]:
# Model
model = 

In [37]:
# Variables definition
x = 

In [38]:
# Objective definition
model.objective = 

In [39]:
# Degree in constraints


# Degree out constraints


# Cutset constraints


In [40]:
# optimizing
model.optimize(relax= True)

<OptimizationStatus.OPTIMAL: 0>

In [41]:
model.objective.x

137.0

In [42]:
# Print the optimal solution

def representation(model,n):
  result = np.zeros((n,n))
  for i in model.vars:
    if len(i.name)>1:
      result[string.ascii_lowercase.index(i.name[0]),string.ascii_lowercase.index(i.name[1])] = i.x
  data = pd.DataFrame(result)
  data.columns = V
  data.index = V
  return data

print(representation(model,n))

     a    b    c    d    e    f
a  0.0  0.0  0.0  0.0  0.0  1.0
b  0.0  0.0  1.0  0.0  0.0  0.0
c  1.0  0.0  0.0  0.0  0.0  0.0
d  0.0  1.0  0.0  0.0  0.0  0.0
e  0.0  0.0  0.0  1.0  0.0  0.0
f  0.0  0.0  0.0  0.0  1.0  0.0


b) Compare the optimal solutions of $(DFJ)$ and $(DFJ^0)$. What do you observe?

2) **Compact extended ILP formulation.**

Consider the partial ILP formulation for the ATSP with only the assignment constraints:

\begin{array}{lll}
   &  \min & \sum\limits_{(i,j) \in A} c_{ij}x_{ij} \qquad & (11) & \\
  & \textrm{s.t.} &  & \\
  & \sum\limits_{j\in V: j \neq i } x_{ij} = 1 &  i \in V & (12) &\\
  & \sum\limits_{i\in V: j \neq i } x_{ij} = 1 &  j \in V & (13) &\\
  & x_{ij} \in \{0,1\} &  (i,j) \in A & (10) &\\
\end{array}

c) Starting from the partial formulation (11)-(14), propose a compact extended formulation for ATSP. Hint: add, for each $i \in V$, an integer variable $t_i$ representing the "position" in
which node $i$ is visited in the tour, and an appropriate set of constraints used to prevent all possible subtours.


d) Explain why such additional constraints exclude any subtour.


If a solution of (15)-(22) contained multiple subtours, then at least one of them would not contain node 1. If such a subtour, say $C$, contains $l$ arcs, summing the constraints (20) associated to these arcs we would obtain
$\sum\limits_{i \in C} t_i \geq \sum\limits_{i \in C} t_i + 1$
which is a contradiction.


e) Is the integrality of variables $t_i$ required?

g) Compare the linear relaxation bound obtained with $(MTZ^0)$ with that obtained with
formulation $(DFJ^0)$. What do you observe?

In [19]:
# SET DEFINITION

# Set of nodes
g = pd.read_csv('V.csv',sep=';')
V = list(g['index'])

# Cardinality of set V
n = len(V)


# PARAMETER DEFINITION

# Cost of each edge
temp_c = pd.read_csv('c.csv',sep=';')
temp_c['index'] = temp_c['index'].apply(ast.literal_eval)
temp_c.set_index('index', inplace = True)
c = temp_c.to_dict()['c']

# Auxiliary set A

A = list(c.keys())

first_V = V[0]


In [27]:
model1 =

In [28]:
x =  
t =  

In [29]:
model1.objective =  

In [30]:
# Degree in constraints
 

# Degree out constraints
 

# first node position
 

# Node position
 

# Tour position
 

In [ ]:
# optimizing
model1.optimize(relax= True)

In [32]:
model1.objective.x

20.2

In [34]:
# Print the optimal solution

def representation(model,n):
  result = np.zeros((n,n))
  for i in model.vars:
    if len(i.name)>1:
      result[string.ascii_lowercase.index(i.name[0]),string.ascii_lowercase.index(i.name[1])] = i.x
  data = pd.DataFrame(result)
  data.columns = V
  data.index = V
  return data

print(representation(model1,n))

     a    b    c    d    e    f
a  0.0  0.0  1.0  0.0  0.0  0.0
b  1.0  0.0  0.0  0.0  0.0  0.0
c  0.0  1.0  0.0  0.0  0.0  0.0
d  0.0  0.0  0.0  0.0  0.2  0.8
e  0.0  0.0  0.0  0.8  0.0  0.2
f  0.0  0.0  0.0  0.2  0.8  0.0
